In [12]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
sys.path.append("..")
import utility.utility_functions as ut
import 


In [6]:
path = r'C:\Users\barka\OneDrive\Final Project\Data\Manhattan\2019'
less_than_10_df = pd.read_csv(path+ r'\post_data_prep_all_zones.csv')


In [7]:
less_than_10_df.head()

,pickup_date,weekday,time_binned,Tmax,Tmin,Tavg,Tdep,HDD,CDD,Precipitation,new_snow,snow_depth,trip_distance,trip_time,speed,num_of_taxis,Zone
0,2019-01-01,Tuesday,00:00 - 00:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.543171,11.890650,12.167499,41,4
1,2019-01-01,Tuesday,01:00 - 01:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.842436,14.737821,11.223936,78,4
2,2019-01-01,Tuesday,02:00 - 02:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.235484,11.183333,11.346154,62,4
3,2019-01-01,Tuesday,03:00 - 03:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.640308,11.701795,12.651760,65,4
4,2019-01-01,Tuesday,04:00 - 04:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.635000,11.158889,13.186688,30,4


In [13]:
main_df = less_than_10_df
df = ut.get_dummies(['weekday', 'time_binned'], main_df)
df = ut.swap_first_and_last_col(df)

In [22]:
df.dtypes

Zone               int64
00:00 - 00:59      uint8
01:00 - 01:59      uint8
02:00 - 02:59      uint8
03:00 - 03:59      uint8
04:00 - 04:59      uint8
05:00 - 05:59      uint8
06:00 - 06:59      uint8
07:00 - 07:59      uint8
08:00 - 08:59      uint8
09:00 - 09:59      uint8
10:00 - 10:59      uint8
11:00 - 11:59      uint8
12:00 - 12:59      uint8
13:00 - 13:59      uint8
14:00 - 14:59      uint8
15:00 - 15:59      uint8
16:00 - 16:59      uint8
17:00 - 17:59      uint8
18:00 - 18:59      uint8
19:00 - 19:59      uint8
20:00 - 20:59      uint8
21:00 - 21:59      uint8
22:00 - 22:59      uint8
23:00 - 23:59      uint8
Friday             uint8
Monday             uint8
Saturday           uint8
Sunday             uint8
Thursday           uint8
Tuesday            uint8
Wednesday          uint8
Tmax             float64
Tmin             float64
Tavg             float64
Tdep             float64
HDD              float64
CDD              float64
Precipitation    float64
new_snow         float64


In [21]:
df.drop('pickup_date',axis=1, inplace=True)

In [14]:
df.head()

,Zone,00:00 - 00:59,01:00 - 01:59,02:00 - 02:59,03:00 - 03:59,04:00 - 04:59,05:00 - 05:59,06:00 - 06:59,07:00 - 07:59,08:00 - 08:59,...,Tdep,HDD,CDD,Precipitation,new_snow,snow_depth,trip_distance,trip_time,speed,num_of_taxis
0,4,1,0,0,0,0,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.543171,11.890650,12.167499,41
1,4,0,1,0,0,0,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.842436,14.737821,11.223936,78
2,4,0,0,1,0,0,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.235484,11.183333,11.346154,62
3,4,0,0,0,1,0,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.640308,11.701795,12.651760,65
4,4,0,0,0,0,1,0,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.635000,11.158889,13.186688,30


In [24]:

y = df.num_of_taxis
X = df[df.columns.tolist()[:-1]]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [26]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [27]:
xg_reg.fit(X_train,y_train)
train_pred = xg_reg.predict(X_train)
test_pred = xg_reg.predict(X_test)


C:\Users\barka\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[11:21:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


In [28]:
rmse_test = np.sqrt(mean_squared_error(y_test, test_pred))
rmse_train = np.sqrt(mean_squared_error(y_train,train_pred))
print("RMSE: %f" % (rmse_test))
print("RMSE: %f" % (rmse_train))

RMSE: 114.141134
RMSE: 114.626581


In [29]:
std_y  =  np.std(y)
std_y

134.699648566846